In [1]:
# import dependencies
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix, balanced_accuracy_score
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.combine import SMOTEENN

In [2]:
# read in our cleaned, merged data
ML_df = pd.read_csv('../resources/superstore_topcountries_ML_prepped.csv', index_col=[0])
ML_df

,Sales,Quantity,Discount,Profit,Shipping Cost,Returned,Profit Margin Percentage,Returned $ Amount,Days to Ship,Orders per Country,...,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables,Order Priority_Critical,Order Priority_High,Order Priority_Low,Order Priority_Medium
0,221.98,2,0.0,62.15,40.770,0.0,27.998018,0.0,2,9994,...,0,0,1,0,0,0,0,1,0,0
1,3709.40,9,0.1,-288.77,923.630,0.0,-7.784817,0.0,2,2837,...,0,0,0,0,0,0,1,0,0,0
2,5175.17,9,0.1,919.97,915.490,0.0,17.776614,0.0,1,2837,...,0,0,1,0,0,0,0,0,0,1
3,2892.51,5,0.1,-96.54,910.160,0.0,-3.337586,0.0,2,2063,...,0,0,1,0,0,0,0,0,0,1
4,2862.68,5,0.1,763.28,897.350,0.0,26.663127,0.0,3,2837,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38609,18.64,1,0.0,8.00,1.010,0.0,42.918455,0.0,3,614,...,0,1,0,0,0,0,0,0,0,1
38610,16.72,5,0.2,3.34,1.930,0.0,19.976077,0.0,4,9994,...,0,0,0,0,0,0,0,1,0,0
38611,8.56,2,0.0,2.48,1.580,0.0,28.971963,0.0,5,9994,...,0,0,0,0,0,0,0,1,0,0
38612,13.44,2,0.0,2.40,1.003,0.0,17.857143,0.0,4,1593,...,0,0,0,0,0,0,0,0,0,1


# Oversampling

In [3]:
# set target (Returned) and features
y = ML_df['Returned']
X = ML_df.drop(columns=['Returned', 'Returned $ Amount'])

In [4]:
# split into train & test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
                                                    #stratify=y)
Counter(y_train)

Counter({0.0: 27635, 1.0: 1325})

## Random Oversampling

In [5]:
# randomly oversample the minority (returned) class
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X_train, y_train)
Counter(y_ros)

Counter({0.0: 27635, 1.0: 27635})

In [6]:
# initiate new model & fit to the oversampled train data
model= LogisticRegression()
model.fit(X_ros, y_ros)

C:\Users\rachm\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [7]:
# create predictions and confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[4713, 4528],
       [ 216,  197]], dtype=int64)

In [8]:
# balanced accuracy score
ros_acc_score = balanced_accuracy_score(y_test, y_pred)
print(f'The balanced accuracy score for the randomly oversampled logistic regression is {ros_acc_score}')

The balanced accuracy score for the randomly oversampled logistic regression is 0.4935036589491038


## Synthetic Minority Oversampling Technique (SMOTE)

In [9]:
# SMOTE the train data sets
X_SMOTE, y_SMOTE = SMOTE(sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_SMOTE)

Counter({0.0: 27635, 1.0: 27635})

In [10]:
# initiate new model & fit to the randomly oversampled train data
model= LogisticRegression()
model.fit(X_SMOTE, y_SMOTE)

C:\Users\rachm\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [11]:
# create predictions and confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[5250, 3991],
       [ 231,  182]], dtype=int64)

In [12]:
# balanced accuracy score
SMOTE_acc_score = balanced_accuracy_score(y_test, y_pred)
print(f'The balanced accuracy score for the SMOTE logistic regression is {SMOTE_acc_score}')

The balanced accuracy score for the SMOTE logistic regression is 0.5043991496994785


## Random Undersampling

In [13]:
# random under sample the train data sets
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X_train, y_train)
Counter(y_rus)

Counter({0.0: 1325, 1.0: 1325})

In [14]:
# initiate new model & fit to the randomly oversampled train data
model= LogisticRegression()
model.fit(X_rus, y_rus)

C:\Users\rachm\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [15]:
# create predictions and confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[3929, 5312],
       [ 174,  239]], dtype=int64)

In [16]:
# balanced accuracy score
rus_acc_score = balanced_accuracy_score(y_test, y_pred)
print(f'The balanced accuracy score for the randomly under sampled logistic regression is {rus_acc_score}')

The balanced accuracy score for the randomly under sampled logistic regression is 0.5019314650233602


## Cluster Centroid Undersampling

In [17]:
# initiate resampling module & use to resample data
cc = ClusterCentroids()
X_cc, y_cc = cc.fit_resample(X_train, y_train)
#Counter(y_cc)

In [18]:
# initiate new model & fit to the undersampled train data
model= LogisticRegression()
model.fit(X_cc, y_cc)

C:\Users\rachm\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [19]:
# create predictions and confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 981, 8260],
       [  37,  376]], dtype=int64)

In [20]:
# balanced accuracy score
cc_acc_score = balanced_accuracy_score(y_test, y_pred)
print(f'The balanced accuracy score for the ClusterCentroid logistic regression is {cc_acc_score}')

The balanced accuracy score for the ClusterCentroid logistic regression is 0.5082844822775016


# SMOTE + Edited Nearest Neighbors (SMOTEENN)

In [21]:
# create instance of SMOTEENN and resample train data 
smote_enn = SMOTEENN()
X_SMOTEENN, y_SMOTEENN = smote_enn.fit_resample(X_train, y_train)

In [22]:
# initiate new model & fit to the resampled train data
model= LogisticRegression()
model.fit(X_SMOTEENN, y_SMOTEENN)

C:\Users\rachm\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [23]:
# create predictions and confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[5000, 4241],
       [ 210,  203]], dtype=int64)

In [24]:
# balanced accuracy score
SMOTEENN_acc_score = balanced_accuracy_score(y_test, y_pred)
print(f'The balanced accuracy score for the SMOTEENN sampled logistic regression is {SMOTEENN_acc_score}')

The balanced accuracy score for the SMOTEENN sampled logistic regression is 0.5162962039107222


# Comparative Results

In [25]:
print(f'The balanced accuracy score for the randomly oversampled logistic regression is {ros_acc_score}')
print(f'The balanced accuracy score for the SMOTE logistic regression is {SMOTE_acc_score}')
print(f'The balanced accuracy score for the randomly under sampled logistic regression is {rus_acc_score}')
print(f'The balanced accuracy score for the ClusterCentroid logistic regression is {cc_acc_score}')
print(f'The balanced accuracy score for the SMOTEENN sampled logistic regression is {SMOTEENN_acc_score}')

The balanced accuracy score for the randomly oversampled logistic regression is 0.4935036589491038
The balanced accuracy score for the SMOTE logistic regression is 0.5043991496994785
The balanced accuracy score for the randomly under sampled logistic regression is 0.5019314650233602
The balanced accuracy score for the ClusterCentroid logistic regression is 0.5082844822775016
The balanced accuracy score for the SMOTEENN sampled logistic regression is 0.5162962039107222
